## The Walks of Nathan Lowell
A look at Nate's path to the back gate.

The journey begins [#tommw](https://www.nathanlowell.com/tommw/)

Archive File Structure

``` python
{'meta': {'ids':[
                '12345',
                '67890',
                ...]
          'last_updated': 'UTC Timestamp',
         }
 'data': [
     {'link':URl string,
      'condition': string,
      ...}
         ]
}
 

```


In [1]:
from io import BytesIO
import json
from PIL import Image
import shutil
import sys

import arrow
from bs4 import BeautifulSoup
import requests


import photo as pto
import podcast as pc
import twitter as twit

[#tommw](https://www.nathanlowell.com/tommw/) is a cross channel social media campaign architeched and executed by science fiction author Nathan Lowell. 

There are 4 planks to Nathan's campain:
1. A blog hosted at https://www.nathanlowell.com/tommw/
2. A photo stream hosted on [FlickR](https://www.flickr.com/photos/nlowell/)
3. Podcast feed via [RSS](http://www.nathanlowell.com/tommw/feed/)
4. Posts to twitter with the #tommw hastag on [@nlowell on twitter](https://twitter.com/nlowell)

In [2]:
# Functions to manage the archive file

def get_archive():

    # Load the archive file
    try:
        with open('archive.json', 'r') as file:
            archive= json.loads(file.read())
        print('Archive loaded.')
        
    except:
        # Create archive data structure.

        archive = {'meta':{'ids':[],
                           'audio':[],
                           'tw_ids':[],
                          'last_updated':'New File',
                          },
                  'data':[],
                  }
        print('No archived file, building new archive.json')
        
    return archive

def save_archive(archive):
    
    with open('archive.json', 'w') as file:
        archive['meta']['last_updated'] = str(arrow.now())
        file.write(json.dumps(archive, indent=4))
        
    return True


def get_tweet_archive(record='tweets'):
    
    filename = record + '.json'
    
    try:
        with open(filename, 'r') as file:
            archive= json.loads(file.read())
        print('tweets_loaded.')
        
    except:
        
        print('No archived tweet file: ', filename,
              ' build a new file with build_twitter_archive function')
        
    return archive



def backfill_dates(archive):
    '''Helper function to convert timestamps
    to a date. Date is used to match audio files to photos.'''

    for walk in archive['data']:

        timestamp = walk['timestamp']

        date = timestamp.split(' ')[0]

        walk['date'] = date

    return archive

    

In [3]:
# Future Main

# Path to temp directory
latest = 'latest/'

# Import the archive from disk
archive = get_archive()
walks = get_tweet_archive('walk')

# Scrape the blog for links to the other media components
blog_page = requests.get("https://www.nathanlowell.com/tommw/")
soup = BeautifulSoup(blog_page.text, 'html.parser')

# Find a flickr link
flickr = soup.find_all(href=True, attrs={'data-flickr-embed':'true'})

# Find a podcast link
try:
    podcast_link = soup.find(class_="powerpress_link_pinw").get('href')

except:
    print('No new walk posts found.')
    podcast_link = None

if not flickr:
    print('No flickr links found on blog home page.')
    flickr_link = None
    
else:
    flickr_link = flickr[0]['href']

    
if not podcast_link and not flickr_link:
    print('No recent walks found on blog.')
    
    
else:
      

    if len(archive['meta']['ids']) < 400:
        album_url = 'https://www.flickr.com/photos/nlowell/albums/72157626736309035'
        archive = pto.build_flickr_archive(flickr_link, archive, album=False)
        archive = pto.build_flickr_archive(album_url, archive, album=True)


    # Process the photostream
    stream_url = pto.get_stream_url(flickr_link)
    stream = pto.get_photostream(stream_url)
    archive = pto.process_photo_stream_page(stream, archive)

    # Check the album page
    album_url = 'https://www.flickr.com/photos/nlowell/albums/72157626736309035'
    stream = pto.get_photostream(album_url, album=True)
    archive = pto.process_photo_stream_page(stream, archive)


    # Save the audio file
    mp3_file, archive = pc.get_mp3(podcast_link, archive)
    
    # Check twitter
    

    # Save the data to archive.json
    save_archive(archive)
 

Archive loaded.
tweets_loaded.
No new walk posts found.
No flickr links found on blog home page.
No recent walks found on blog.


In [8]:
tweets = twit.new_tweets(archive, start_date=datetime.date(2019,1,1))

INFO: queries: ['#tommw since:2019-01-01 until:2019-01-02', '#tommw since:2019-01-02 until:2019-01-03', '#tommw since:2019-01-03 until:2019-01-05', '#tommw since:2019-01-05 until:2019-01-06', '#tommw since:2019-01-06 until:2019-01-07', '#tommw since:2019-01-07 until:2019-01-09', '#tommw since:2019-01-09 until:2019-01-10', '#tommw since:2019-01-10 until:2019-01-11', '#tommw since:2019-01-11 until:2019-01-13', '#tommw since:2019-01-13 until:2019-01-14', '#tommw since:2019-01-14 until:2019-01-15', '#tommw since:2019-01-15 until:2019-01-17', '#tommw since:2019-01-17 until:2019-01-18', '#tommw since:2019-01-18 until:2019-01-19', '#tommw since:2019-01-19 until:2019-01-21', '#tommw since:2019-01-21 until:2019-01-22', '#tommw since:2019-01-22 until:2019-01-23', '#tommw since:2019-01-23 until:2019-01-25', '#tommw since:2019-01-25 until:2019-01-26', '#tommw since:2019-01-26 until:2019-01-27', '#tommw since:2019-01-27 until:2019-01-29', '#tommw since:2019-01-29 until:2019-01-30', '#tommw since:20

In [ ]:
tweets
filtered = twit.filter_twitter_search(tweets)
filtered